# Lab 7

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_07.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

## Datasets

The datasets being used in the lab are listed below:

- [TIGER: US Census Counties](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties)
- [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)
- [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
- [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ)

In [1]:
import ee 
import geemap
ee.Initialize()

## Question 1

Create a fishnet with a 4-degree interval based on the extent of `[-112.5439, 34.0891, -85.0342, 49.6858]`. Use the fishnet to download the Landsat 7 image tiles by the fishnet using the `geemap.download_ee_image_tiles()` function. Relevant Earth Engine assets:

-   `ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')`

![](https://i.imgur.com/L1IH3fq.png)

In [7]:
m= geemap.Map(center=[40,-100], zoom=4)
image= ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(['B4', 'B3', 'B2'])

vis_params= {
    'min':0,
    'max':255,
    'gamma': [0.95, 1.1, 1]
}
m.addLayer(image, vis_params, 'Landsat')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [14]:
region= ee.Geometry.BBox(-112.5439, 34.0891, -85.0342, 49.6858)
fc = ee.FeatureCollection(region)
fishnet=geemap.fishnet(fc.geometry(), rows=4, cols=7, delta=0.5)
style={'color':'ffff00ff', 'fillColor':'00000000'}
m.addLayer(fishnet.style(**style), {}, 'Fishnet')

In [15]:
outdir= 'tiles'
geemap.download_ee_image_tiles(
    image,
    fishnet,
    outdir,
    prefix='Landsat_',
    crs='EPSG:4326',
    scale=120
)

Landsat_01.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Landsat_02.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Landsat_03.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Landsat_04.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Landsat_05.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

...

Landsat_25.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Landsat_26.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Landsat_27.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Landsat_28.tif: |          | 0.00/39.6M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 28 tiles in 188.30196857452393 seconds.


## Question 2

Create annual cloud-free Landsat imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer. 

![](https://i.imgur.com/MN2UXHx.png)

In [ ]:
# Add your code here.

## Question 3

Create annual cloud-free Sentinel-2 imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process. Narrow down the date range (e.g., summer months) to reduce the number of images, which can avoid memory errors.

![](https://i.imgur.com/r5RQlEJ.png)

In [ ]:
# Add your code here.

## Question 4

Create annual cloud-free NAIP imagery for the years 2010-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process. 

![](https://i.imgur.com/h66FC8h.png)

In [ ]:
# Add your code here.

## Question 5

Download a US county of your choice and save it as a shapefile or GeoJSON file. 

![](https://i.imgur.com/PuK2Vp3.png)

In [ ]:
# Add your code here.